# Comment optimiser son code ?

- trouver les régions lentes du code en faisant du profiling
- écrire sa propre API C/Python (si vous en avez le courage !!!!)
- optimiser ces parties en se ramenant à un langage bas niveau (Cython, Pythran, Numba, ...)

# Calcul de $\pi$
On souhaite approcher $\pi$ en calculant l'intégrale suivante

$$
\pi = \int_0^1f(x)dx \; \text{avec} \; f(x)=\frac{4}{1+x^2}.
$$

Pour ce faire, on approche l'intégrale en utilisant une méthode des rectangles

$$
\pi \approx \frac{1}{n}\sum_{i=0}^{n-1}f(x_i), \; \text{avec} \; x_i=\frac{i+0.5}{n} \; \text{pour} \; i=0,\cdots,n-1.
$$

Voici une écriture possible en Python

In [ ]:
def f(x):
    return 4./(1 + x**2)

def calculPi(n):
    h, pi = 1./n, 0.
    for i in xrange(n):
        pi += f(h*(i+.5))
    return h*pi

## Outils de profiling

### Le module `time`
Une première façon de profiler son code est d'utiliser le module `time` de Python.

In [ ]:
import time
n, nrep = 1000000, 10

t1 = time.time()
for i in xrange(nrep):
    pi = calculPi(n)
print pi, (time.time() - t1)/nrep

### Le module `timeit`

On peut appeler de base `timeit` avec le notebook de IPython via la commande magic `%timeit`. Dans un script Python, il est nécessaire de faire

```
from timeit import timeit
```

In [ ]:
%timeit calculPi(n)

### Le module `cProfile`

`cProfile` est fourni de base avec Python et est écrit en C. Il permet d'avoir des informations basiques de profiling.

In [ ]:
import cProfile

cProfile.run('calculPi(%d)'%n)

### le module `line_profiler`

Ce module est beaucoup plus complet que les précedents puisqu'il est possible d'avoir le profiling du code ligne à ligne.

On peut profiler n'importe quelle fontion de notre code en rajoutant le decorateur `profile` et en utilisant le script `kernprof`.

Reprenons notre exemple en ajoutant le décorateur.

In [ ]:
%%file calculPi.py
@profile
def f(x):
    return 4./(1 + x**2)

@profile
def calculPi(n):
    h, pi = 1./n, 0.
    for i in xrange(n):
        pi += f(h*(i+.5))
    return h*pi

if __name__ == '__main__':
    n = 1000000
    calculPi(n)

In [ ]:
!kernprof -l -v calculPi.py

## Pythran

### Principe général

- se branche sur le module écrit en Python
- ajout d'un commentaire permettant d'indiquer les fonctions à optimiser
- construction d'un graphe de flot de contrôle
- recherche des types des variables par inférence
- optimisation des fonctions

<center>
<img src="files/pythrandiag1.png" style="width: 100%;" />
</center>

Nous allons utiliser Pythran directement dans le notebook en utilisant la commande `%load_ext`.

In [ ]:
import pythran
%load_ext pythran.magic

In [ ]:
%%pythran

#pythran export calculPi_pythran(int)

def f(x):
    return 4./(1 + x**2)

def calculPi_pythran(n):
    h, pi = 1./n, 0.
    for i in range(n):
        pi += f(h*(i+.5))
    return h*pi

In [ ]:
n=100000000
print calculPi_pythran(n)
%timeit calculPi_pythran(n)

On peut facilement ajouter des pragmas openMP.

In [ ]:
%%pythran -fopenmp

#pythran export calculPi_pythran_omp(int)

def f(x):
    return 4./(1 + x**2)

def calculPi_pythran_omp(n):
    h, pi = 1./n, 0.
    #omp parallel for reduction(+:pi)
    for i in range(n):
        pi += f(h*(i+.5))
    return h*pi

In [ ]:
print calculPi_pythran_omp(n)
%timeit calculPi_pythran_omp(n)

# Modèle de Gray-Scott

Le modèle de Gray-Scott permet de simuler deux espèces chimiques $A$ et $B$ qui réagissent entre elles et qui diffusent.

Le modèle mathématique est assez simple

$$
\begin{array}{l}
\frac{\partial A}{\partial t} = D_A \Delta A - AB^2 + f(1-A) \\
\frac{\partial B}{\partial t} = D_B \Delta B + AB^2 - (k+f)B
\end{array}
$$

On peut approcher le laplacien par un schéma aux différences finies à 5 points. 

$$
\Delta A_{i,j} \approx A_{i,j-1} + A_{i-1,j} -4A_{i,j} + A_{i+1, j} + A_{i, j+1}
$$

La dérivée en temps peut être approchée par un schéma d'Euler explicite.

## Initialisation

$A$ vaut $1$ partout et $B$ vaut $0$ partout sauf à un endroit du domaine où on met $0.25$. Cet endroit sera une ellipse centrée au centre du domaine $[0, 1]\times[0,1]$et de rayons $r_x=0.02$ et $r_y=0.04$.

In [ ]:
import numpy as np

def init(n):
    A = np.ones((n+2,n+2))
    B = np.zeros((n+2,n+2))

    x = np.linspace(0, 1, n+2)
    y = np.linspace(0, 1, n+2)[:, np.newaxis]

    rx = .02
    ry = .04

    mask = (x-.5)**2/rx**2 + (y-.5)**2/ry**2 < 1
    B[mask] = .25
    return A, B

## Conditions périodiques

In [ ]:
def periodic_cond(u):
    u[0, :] = u[-2, :]
    u[-1, :] = u[1, :]
    u[:, 0] = u[:, -2]
    u[:, -1] = u[:, 1]

## Laplacien et modèle de Gray-Scott

In [ ]:
def laplacian(u):
  return (                  u[ :-2, 1:-1] +
           u[1:-1, :-2] - 4*u[1:-1, 1:-1] + u[1:-1, 2:] +
                        +   u[2:  , 1:-1] )

def grayscott(A, B, Da, Db, f, k):
    a, b = A[1:-1,1:-1], B[1:-1,1:-1]

    La = laplacian(A)
    Lb = laplacian(B)

    abb = a*b*b
    a += Da*La - abb + f*(1 - a)
    b += Db*Lb + abb - (f + k)*b

In [ ]:
def animate(i):
  global A, B, Da, Db; f, k
  for t in range(50):
    grayscott(A, B, Da, Db, f, k)
    periodic_cond(A)
    periodic_cond(B)
  im.set_array(B)
  return im,

In [ ]:
Da, Db = .1, .05
f, k = 0.0367, 0.0649
#f, k = 0.0545, 0.062
#f, k = 0.018, 0.050
#f, k = 0.050, 0.065
#f, k = 0.035, 0.060

In [ ]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.cm as cm

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

A, B = init(200)

im = plt.imshow(B, cmap = cm.Greys)

anim = animation.FuncAnimation(fig, animate, np.arange(1, 200),
    interval=2, blit=True)

plt.show()

In [ ]:
A, B = init(2000)
%timeit grayscott(A, B, Da, Db, f, k)

## Cython

Nous allons écrire le code Cython associé à ce problème afin d'avoir un temps de référence.

Pour ce faire, on utilise la commande magic `cython` pour pouvoir utiliser directement Cython via le notebook. L'option `-a` permet de voir le code via l'API C/Python généré.

In [ ]:
%load_ext cython

In [ ]:
%%cython -a --compile-args=-fopenmp --link-args=-fopenmp
cimport cython
from cython.parallel import prange, parallel
import numpy as np

@cython.boundscheck(False)
def grayscott_cython(double[:, ::1] A, double[:, ::1] B, double Da, double Db, double f, double k):
    cdef:
        int i, j, itime
        int nx = A.shape[0]
        int ny = A.shape[1]
        double[:, ::1] La = np.zeros_like(A)
        double[:, ::1] Lb = np.zeros_like(A)
        double abb
          
    for i in range(1, nx - 1):
        for j in range(1, ny - 1):
            La[i, j] = A[i-1, j] + A[i, j-1] - 4*A[i, j] + A[i+1, j] + A[i, j+1]
            Lb[i, j] = B[i-1, j] + B[i, j-1] - 4*B[i, j] + B[i+1, j] + B[i, j+1]

    for i in range(1, nx - 1):
        for j in range(1, ny - 1):
            abb = A[i, j]*B[i, j]*B[i, j]
            A[i, j] += Da*La[i, j] - abb + f*(1 - A[i, j])
            B[i, j] += Db*Lb[i, j] + abb - (f + k)*B[i, j]
    
@cython.boundscheck(False)
def grayscott_cython_openmp(double[:, ::1] A, double[:, ::1] B, double Da, double Db, double f, double k):
    cdef:
        int i, j, itime
        int nx = A.shape[0]
        int ny = A.shape[1]
        double[:, ::1] La = np.zeros_like(A)
        double[:, ::1] Lb = np.zeros_like(A)
        double abb
          
    with nogil, parallel(num_threads=8):
        for i in prange(1, nx - 1, schedule='dynamic'):
            for j in range(1, ny - 1):
                La[i, j] = A[i-1, j] + A[i, j-1] - 4*A[i, j] + A[i+1, j] + A[i, j+1]
                Lb[i, j] = B[i-1, j] + B[i, j-1] - 4*B[i, j] + B[i+1, j] + B[i, j+1]

        for i in prange(1, nx - 1, schedule='dynamic'):
            for j in range(1, ny - 1):
                abb = A[i, j]*B[i, j]*B[i, j]
                A[i, j] += Da*La[i, j] - abb + f*(1 - A[i, j])
                B[i, j] += Db*Lb[i, j] + abb - (f + k)*B[i, j]

In [ ]:
A, B = init(2000)
%timeit grayscott_cython(A, B, Da, Db, f, k)

In [ ]:
A, B = init(2000)
%timeit grayscott_cython_openmp(A, B, Da, Db, f, k)

## Pythran

In [ ]:
%%pythran -fopenmp
import numpy as np
def laplacian(u):
  return (                  u[ :-2, 1:-1] +
           u[1:-1, :-2] - 4*u[1:-1, 1:-1] + u[1:-1, 2:] +
                        +   u[2:  , 1:-1] )

#pythran export grayscott_pythran(float64[][], float64[][], float64, float64, float64, float64)
#pythran export grayscott_pythran_unroll(float64[][], float64[][], float64, float64, float64, float64)
#pythran export grayscott_pythran_unroll_omp(float64[][], float64[][], float64, float64, float64, float64)

def grayscott_pythran(A, B, Da, Db, f, k):
    a, b = A[1:-1,1:-1], B[1:-1,1:-1]

    La = laplacian(A)
    Lb = laplacian(B)

    abb = a*b*b
    a += Da*La - abb + f*(1 - a)
    b += Db*Lb + abb - (f + k)*b

def grayscott_pythran_unroll(A, B, Da, Db, f, k):
    a, b = A[1:-1,1:-1], B[1:-1,1:-1]
    La = np.zeros_like(A)
    Lb = np.zeros_like(A)

    for i in range(1, A.shape[0]-1):
        for j in range(1, A.shape[1]-1):
            La[i, j] = A[i-1, j] + A[i, j-1] - 4*A[i, j] + A[i+1, j] + A[i, j+1]
            Lb[i, j] = B[i-1, j] + B[i, j-1] - 4*B[i, j] + B[i+1, j] + B[i, j+1]

    for i in range(1, A.shape[0]-1):
        for j in range(1, A.shape[1]-1):
            abb = A[i, j]*B[i, j]*B[i, j]
            A[i, j] += Da*La[i, j] - abb + f*(1 - A[i, j])
            B[i, j] += Db*Lb[i, j] + abb - (f + k)*B[i, j]
            
def grayscott_pythran_unroll_omp(A, B, Da, Db, f, k):
    a, b = A[1:-1,1:-1], B[1:-1,1:-1]
    La = np.zeros_like(A)
    Lb = np.zeros_like(A)

    #omp parallel for 
    for i in range(1, A.shape[0]-1):
        #omp parallel for 
        for j in range(1, A.shape[1]-1):
            La[i, j] = A[i-1, j] + A[i, j-1] - 4*A[i, j] + A[i+1, j] + A[i, j+1]
            Lb[i, j] = B[i-1, j] + B[i, j-1] - 4*B[i, j] + B[i+1, j] + B[i, j+1]

    #omp parallel for 
    for i in range(1, A.shape[0]-1):
        #omp parallel for 
        for j in range(1, A.shape[1]-1):
            abb = A[i, j]*B[i, j]*B[i, j]
            A[i, j] += Da*La[i, j] - abb + f*(1 - A[i, j])
            B[i, j] += Db*Lb[i, j] + abb - (f + k)*B[i, j]            

In [ ]:
A, B = init(2000)
%timeit grayscott_pythran(A, B, Da, Db, f, k)

In [ ]:
A, B = init(2000)
%timeit grayscott_pythran_unroll(A, B, Da, Db, f, k)

In [ ]:
A, B = init(2000)
%timeit grayscott_pythran_unroll_omp(A, B, Da, Db, f, k)

## Misc

- spécialiser les fonctions par rapport à différents types
- compatible Python 2 et Python 3
- possibilité de voir le code C++ généré
- possibilité de voir les flags de compilation
- support pour une utilisation avec distutils

## Pour aller plus loin

- [Pythran tutorial](http://serge-sans-paille.github.io/pythran-stories/pythran-tutorial.html)(dont ce notebook s'est fortement inspiré)
- GitHub: https://github.com/serge-sans-paille/pythran
- Mailing list: http://www.freelists.org/list/pythran
- IRC: #pythran on FreeNode
- StackOverflow: http://stackoverflow.com/questions/tagged/pythran


In [43]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()